In [1]:
from analytics import create_imdb_df
import pandas as pd

In [2]:
# from downloader import download_all_imdb_pages
# download_all_imdb_pages()

In [3]:
# df = create_imdb_df()
df = pd.read_csv('imdb.csv', dtype='str', keep_default_na=False)

FileNotFoundError: [Errno 2] No such file or directory: 'imdb.csv'

In [ ]:
df.dtypes

url            object
title          object
year           object
rating         object
runtime        object
certificate    object
genre          object
metascore      object
description    object
cast           object
votes          object
gross          object
dtype: object

In [4]:
df.head()

NameError: name 'df' is not defined

In [67]:
len(df)

9998

In [68]:
# from downloader import download_all_boxoffice_pages

# urls = df['url'].tolist()
# download_all_boxoffice_pages(urls)

In [69]:
from cleanup import clean_spaces, parse_cast, parse_year, parse_votes

for column in df.columns:
    df[column] = df[column].apply(clean_spaces)

### Process cast -> (director, stars)

In [70]:
dirs_stars = df['cast'].apply(parse_cast)

dirs, stars = zip(*dirs_stars)

dirs, stars = list(dirs), list(stars)

df['director'] = dirs
df['stars'] = stars

df = df.drop(columns=['cast'])

### Type coercions

In [71]:
df['year'] = df['year'].apply(parse_year)

In [72]:
df.head()

,url,title,year,rating,runtime,certificate,genre,metascore,description,votes,gross,director,stars
0,11737520,One Piece,2023–,8.5,60 min,TV-14,"Action, Adventure, Comedy",,"In a seafaring world, a young pirate captain s...","80,433",,,"Iñaki Godoy, Emily Rudd, Mackenyu, Vincent Regan"
1,13622776,Ahsoka,2023–,8.0,,TV-14,"Action, Adventure, Drama",,"After the fall of the Galactic Empire, former ...","34,287",,,"Rosario Dawson, Natasha Liu Bordizzo, Mary Eli..."
2,7462410,The Wheel of Time,2021–,7.1,60 min,TV-14,"Action, Adventure, Drama",,Set in a high fantasy world where magic exists...,"117,336",,,"Rosamund Pike, Daniel Henney, Madeleine Madden..."
3,0388629,One Piece,1999–,8.9,24 min,TV-14,"Animation, Action, Adventure",,Follows the adventures of Monkey D. Luffy and ...,"191,111",,,"Mayumi Tanaka, Akemi Okamura, Laurent Vernin, ..."
4,15398776,Oppenheimer,2023,8.6,180 min,R,"Biography, Drama, History",88,"The story of American scientist, J. Robert Opp...","429,410",,Christopher Nolan,"Cillian Murphy, Emily Blunt, Matt Damon, Rober..."


In [73]:
df['votes'] = df['votes'].apply(parse_votes)

In [74]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')
df['gross'] = pd.to_numeric(df['gross'], errors='coerce')

In [75]:
import numpy as np

df['runtime'] = df['runtime'].apply(lambda x: x.replace(',', '').removesuffix(' min') if x else 0)
# df['runtime'] = pd.to_numeric(df['runtime'], downcast="integer", errors='coerce')
df['runtime'] = df['runtime'].astype(np.int64)

In [76]:
df.head()

,url,title,year,rating,runtime,certificate,genre,metascore,description,votes,gross,director,stars
0,11737520,One Piece,2023–,8.5,60,TV-14,"Action, Adventure, Comedy",NaN,"In a seafaring world, a young pirate captain s...",80433,NaN,,"Iñaki Godoy, Emily Rudd, Mackenyu, Vincent Regan"
1,13622776,Ahsoka,2023–,8.0,0,TV-14,"Action, Adventure, Drama",NaN,"After the fall of the Galactic Empire, former ...",34287,NaN,,"Rosario Dawson, Natasha Liu Bordizzo, Mary Eli..."
2,7462410,The Wheel of Time,2021–,7.1,60,TV-14,"Action, Adventure, Drama",NaN,Set in a high fantasy world where magic exists...,117336,NaN,,"Rosamund Pike, Daniel Henney, Madeleine Madden..."
3,0388629,One Piece,1999–,8.9,24,TV-14,"Animation, Action, Adventure",NaN,Follows the adventures of Monkey D. Luffy and ...,191111,NaN,,"Mayumi Tanaka, Akemi Okamura, Laurent Vernin, ..."
4,15398776,Oppenheimer,2023,8.6,180,R,"Biography, Drama, History",88.0,"The story of American scientist, J. Robert Opp...",429410,NaN,Christopher Nolan,"Cillian Murphy, Emily Blunt, Matt Damon, Rober..."


In [77]:
df.dtypes

url             object
title           object
year            object
rating         float64
runtime          int64
certificate     object
genre           object
metascore      float64
description     object
votes            int64
gross          float64
director        object
stars           object
dtype: object

In [78]:
df.to_csv('imdb_clean.csv', index=False)

In [ ]:
from boxoffice import create_boxoffice_df

ids = df['url'].to_list()

# df = create_boxoffice_df(ids)
# df.to_csv('boxoffice.csv', index=False)

In [79]:
df = pd.read_csv('boxoffice.csv', dtype='str', keep_default_na=False)

In [80]:
df.head()

,id,domestic,domestic_percent,international,international_percent,worldwide,domestic_opening,budget,release_date,distributor
0,11737520,,,,,,,,,
1,13622776,,,,,,,,,
2,7462410,,,,,,,,,
3,0388629,,,,,,,,,
4,15398776,"$318,639,115",34.9%,"$594,062,000",65.1%,"$912,701,115","$82,455,420",,"July 19, 2023\n (EMEA, APAC)",Universal PicturesSee full company information


In [89]:
length = len(df)

In [90]:
for col in df.columns:
    empty = df[col].apply(lambda x: False if x == '' else True)
    nans = empty.sum() / length
    print(col, nans)

id 1.0
domestic 0.5663132626525305
domestic_percent 0.5663132626525305
international 0.5663132626525305
international_percent 0.5663132626525305
worldwide 0.39797959591918386
domestic_opening 0.45609121824364873
budget 0.22694538907781556
release_date 0.5858171634326865
distributor 0.5273054610922184


# Затем можно сделать Join. Однако удобнее оставить данные двумя датасетами.